<a href="https://colab.research.google.com/github/smaciasg/Ejercicios_DataScience_Coding_Dojo/blob/main/Actividad_de_pipelines_(base).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Para esta tarea, usaran el conjunto de datos de cereales. Este conjunto de datos muestra cereales populares por marca y fabricante junto con información nutricional.  La pregunta de aprendizaje automático es: 

¿Qué tan bien se pueden predecir las calorías basándose en el fabricante, tipo de cereal, gramos de grasa, gramos de azúcares y el peso en onzas por una porción del cereal? 

#**PARTE 1**

Deberán:

Definir la característica (X) y objetivo (y).

Utilizar train test split en los datos para prepararlos para aprendizaje automático.

Identificar cada característica como numérica, ordinal o nominal. (Entreguen esta pregunta en una celda de texto en un cuaderno de Colab).

Utilizar pipelines y transformadores de columnas para completar las siguientes tareas:

Imputar algún valor faltante. Utilizar la estrategia “mean” para las columnas numéricas y la estrategia “most_frequent” para las columnas categóricas.

Realizar una codificación one-hot a las características nominales.

Escalar las columnas numéricas.

Todos los datos de preprocesamiento deben estar contenidos en un único objeto de preprocesamiento.

Utilizar el objeto de preprocesamiento para transformar los datos de manera adecuada, evitar la fuga de datos para dejarlos listos para el modelado. 

Muestren los arrays de NumPy resultantes.

####**1. Importar las bibliotecas necesarias**

In [19]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn import set_config
set_config(display='diagram')

####**2) Importar los datos y explorarlos**

In [20]:
cereales = pd.read_csv('/content/drive/MyDrive/CODING DOJO/DATA SCIENCE/ACTIVIDADES SEMANA 5 - INTRO APRENDIZAJE AUTOMÁTICO ML/Cereal with missing values.xlsx - Sheet 1 - cereal.csv')
cereales.head()

,name,Manufacturer,type,calories per serving,grams of protein,grams of fat,milligrams of sodium,grams of dietary fiber,grams of complex carbohydrates,grams of sugars,milligrams of potassium,vitamins and minerals (% of FDA recommendation),Display shelf,Weight in ounces per one serving,Number of cups in one serving,Rating of cereal
0,Apple Cinnamon Cheerios,General Mills,Cold,110.0,2,2.0,180.0,1.5,10.5,10.0,70,25.0,1,1.00,0.75,29.509541
1,Basic 4,General Mills,Cold,130.0,3,2.0,NaN,2.0,18.0,NaN,100,25.0,3,1.33,0.75,37.038562
2,Cheerios,General Mills,Cold,NaN,6,2.0,290.0,2.0,17.0,1.0,105,25.0,1,1.00,1.25,50.764999
3,Cinnamon Toast Crunch,General Mills,Cold,120.0,1,3.0,210.0,0.0,13.0,9.0,45,25.0,2,1.00,0.75,19.823573
4,Clusters,General Mills,Cold,110.0,3,2.0,140.0,2.0,13.0,7.0,105,25.0,3,1.00,0.50,40.400208


In [21]:
cereales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 16 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   name                                             77 non-null     object 
 1   Manufacturer                                     77 non-null     object 
 2   type                                             68 non-null     object 
 3   calories per serving                             70 non-null     float64
 4   grams of protein                                 77 non-null     int64  
 5   grams of fat                                     69 non-null     float64
 6   milligrams of sodium                             76 non-null     float64
 7   grams of dietary fiber                           77 non-null     float64
 8   grams of complex carbohydrates                   77 non-null     float64
 9   grams of sugars                   

**COLUMNA OBJETIVO: calories per serving**



In [22]:
from pandas.core.internals import create_block_manager_from_arrays
#Columnas con datos faltantes
cereales.isna().sum()

#type - objetc
#calories per serving - float
#grams of fat - float
#milligrams of sodium - float
#grams of sugars - float
#vitamins and minerals (% of FDA recommendation) - float

# Esto quiere decir que solo tenemos que imputar los datos flotantes y un objeto

name                                               0
Manufacturer                                       0
type                                               9
calories per serving                               7
grams of protein                                   0
grams of fat                                       8
milligrams of sodium                               1
grams of dietary fiber                             0
grams of complex carbohydrates                     0
grams of sugars                                    9
milligrams of potassium                            0
vitamins and minerals (% of FDA recommendation)    1
Display shelf                                      0
Weight in ounces per one serving                   0
Number of cups in one serving                      0
Rating of cereal                                   0
dtype: int64

**IDENTIFICACIÓN DE CARACTERÍSTICAS**

 0   name                                             NOMINAL 

 1   Manufacturer                                     NOMINAL 

 2   type                                             NOMINAL 

 3   calories per serving                             NUMÉRICA

 4   grams of protein                                 NUMÉRICA 

 5   grams of fat                                     NUMÉRICA

 6   milligrams of sodium                             NUMÉRICA

 7   grams of dietary fiber                           NUMÉRICA

 8   grams of complex carbohydrates                   NUMÉRICA

 9   grams of sugars                                  NUMÉRICA

 10  milligrams of potassium                          NUMÉRICA 

 11  vitamins and minerals (% of FDA recommendation)  NUMÉRICA

 12  Display shelf                                    NUMÉRICA 

 13  Weight in ounces per one serving                 NUMÉRICA

 14  Number of cups in one serving                    NUMÉRICA
 
 15  Rating of cereal                                 NUMÉRICA

In [23]:
#Ver que tal están escritos los datos categóricos
for i in cereales.columns:
  if cereales[i].dtypes == 'O':
    print(f'{i}:\n{cereales[i].unique()}\n-----------------')

#Como no conozco las marcas supondré que están bien, por los otras dos columnas de candena, no veo que cambiar
#Respecto a columnas ordinales, no veo que alguna de las columnas de cadena deba de tener algún tipo de jerarquía, por lo que no se realiza nungún tipo de cambio

name:
['Apple Cinnamon Cheerios' 'Basic 4' 'Cheerios' 'Cinnamon Toast Crunch'
 'Clusters' 'Cocoa Puffs' 'Count Chocula' 'Crispy Wheat & Raisins'
 'Golden Grahams' 'Honey Nut Cheerios' 'Kix' 'Lucky Charms'
 'Multi-Grain Cheerios' 'Oatmeal Raisin Crisp' 'Raisin Nut Bran'
 'Total Corn Flakes' 'Total Raisin Bran' 'Total Whole Grain' 'Triples'
 'Trix' 'Wheaties' 'Wheaties Honey Gold' 'All-Bran'
 'All-Bran with Extra Fiber' 'Apple Jacks' 'Corn Flakes' 'Corn Pops'
 "Cracklin' Oat Bran" 'Crispix' 'Froot Loops' 'Frosted Flakes'
 'Frosted Mini-Wheats' 'Fruitful Bran' 'Just Right Crunchy  Nuggets'
 'Just Right Fruit & Nut' 'Mueslix Crispy Blend' 'Nut&Honey Crunch'
 'Nutri-Grain Almond-Raisin' 'Nutri-grain Wheat' 'Product 19'
 'Raisin Bran' 'Raisin Squares' 'Rice Krispies' 'Smacks' 'Special K'
 '100% Bran' 'Shredded Wheat' "Shredded Wheat 'n'Bran"
 'Shredded Wheat spoon size' 'Strawberry Fruit Wheats' 'Bran Flakes'
 'Fruit & Fibre Dates; Walnuts; and Oats' 'Fruity Pebbles' 'Golden Crisp'
 'Grape N

####**3) División de la validación: Crear matriz de características y vector objetivo**

In [24]:
X = cereales.drop(columns='calories per serving')
y = cereales['calories per serving']
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

####**4) Crear los selectores de columnas categóricas y numéricas**

In [25]:
numeric_selector = make_column_selector(dtype_include='number')
category_selector = make_column_selector(dtype_include='object')

####**5) Instanciar los transformadores numérico y categórico**

**Transformadores para los IMPUTER**

In [26]:
#Dado que no hay datos faltantes en los datos numéricos de tipo entero, el imputador a usar seré la media
imputer_numeric_flotat = SimpleImputer(strategy='mean')
imputer_object_frequente = SimpleImputer(strategy='most_frequent')

**Transformador de escala - Escalador**

In [27]:
scaler = StandardScaler()

**Tranformador para datos categóricos OneHotEncoder**

In [28]:
ohe = OneHotEncoder(handle_unknown='ignore',sparse=False)

####**6) Crear/instanciar los PIPELINES a usar**

**pipeline numérico**

In [29]:
numeric_pipe = make_pipeline(imputer_numeric_flotat,scaler) #Reciben un imputador y un transformador
numeric_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('standardscaler', StandardScaler())])

**pipeline categórico**

In [30]:
category_pipe = make_pipeline(imputer_object_frequente, ohe) #Recibe un imputador y una transformador 
category_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                ('onehotencoder',
                 OneHotEncoder(handle_unknown='ignore', sparse=False))])

####**7) Instanciar ColumnTransformer**

**Tupla para ColumnTransformer categórico**

In [31]:
category_tuple = (category_pipe, category_selector) #Recibe un pipeline y un selector de columnas
category_tuple

(Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='most_frequent')),
                 ('onehotencoder',
                  OneHotEncoder(handle_unknown='ignore', sparse=False))]),
 <sklearn.compose._column_transformer.make_column_selector at 0x7f4020858710>)

**Tupla para ColumnTransformer numérico**

In [32]:
numeric_tuple = (numeric_pipe, numeric_selector) #Recibe un pipeline y el selector de columnas numéricas
numeric_tuple

(Pipeline(steps=[('simpleimputer', SimpleImputer()),
                 ('standardscaler', StandardScaler())]),
 <sklearn.compose._column_transformer.make_column_selector at 0x7f40208a0950>)

**CREAMOS EL COLUMNTRANSFORMER**

In [33]:
preprocessed = make_column_transformer(numeric_tuple,category_tuple,remainder='passthrough') #Recibe las tuplas categóricas, numérica y una acción para los datos que no están allí contenidos
preprocessed

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f40208a0950>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f4020858710>)])

####**8) Transformador de datos**

**Ajustar la variable que contiene el ColumnTrasnformer con los datos de ENTRENAMIENTO**

In [34]:
preprocessed.fit(X_train)

ColumnTransformer(remainder='passthrough',
                  transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f40208a0950>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f4020858710>)])

**TRANSFORMAR los datos de entrenamiento y los de prueba**

In [35]:
X_train_processed = preprocessed.transform(X_train)
X_train_processed

array([[-1.30301442, -0.97467943,  0.56162348, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.40438378,  0.        ,  0.68120871, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.40438378, -0.97467943,  1.99664622, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 1.25808288,  1.94935887, -0.03630266, ...,  1.        ,
         1.        ,  0.        ],
       [ 0.40438378,  0.97467943, -0.15588789, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.40438378,  0.        ,  0.08328257, ...,  0.        ,
         1.        ,  0.        ]])

In [36]:
X_test_processed = preprocessed.transform(X_test)
X_test_processed

array([[ 0.40438378,  0.97467943, -0.15588789, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.40438378,  0.97467943, -0.03630266, ...,  0.        ,
         1.        ,  0.        ],
       [-0.44931532,  0.        ,  1.27913485, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-1.30301442,  0.        ,  0.32245303, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.40438378,  0.        ,  0.2028678 , ...,  0.        ,
         1.        ,  0.        ],
       [ 1.25808288,  0.        ,  1.27913485, ...,  0.        ,
         1.        ,  0.        ]])

####**9) Inspeccionar el resultado**

In [37]:
print(np.isnan(X_train_processed).sum().sum(), 'Cantidad de datos faltante en matriz de características de entrenamiento')
print(np.isnan(X_test_processed).sum().sum(), 'Cantidad de datos faltantes en la matriz de características de prueba')
print('--------------------------------------------------------------')
print('Todos los datos en el conjunto X_train_processed son del tipo:' , X_train_processed.dtype)
print('Todos los datos en el conjunto X_test_processed son del tipo:', X_test_processed.dtype)
print('--------------------------------------------------------------')
print('La forma de los datos de entrenamiento es: ', X_train_processed.shape)
print('La forma de los datos de prueba es: ', X_test_processed.shape)

0 Cantidad de datos faltante en matriz de características de entrenamiento
0 Cantidad de datos faltantes en la matriz de características de prueba
--------------------------------------------------------------
Todos los datos en el conjunto X_train_processed son del tipo: float64
Todos los datos en el conjunto X_test_processed son del tipo: float64
--------------------------------------------------------------
La forma de los datos de entrenamiento es:  (57, 77)
La forma de los datos de prueba es:  (20, 77)
